## Configuration

In [ ]:
import pyreadr
from sklearn.preprocessing import StandardScaler

TARGET_VARIABLE_COLUMN_NAME = "faultNumber"
SIMULATION_RUN_COLUMN_NAME = "simulationRun"
COLUMNS_TO_REMOVE = ["faultNumber","simulationRun", "sample"]



fault_free_training_dict = pyreadr.read_r("data/TEP_FaultFree_Training.RData")
fault_free_testing_dict = pyreadr.read_r("data/TEP_FaultFree_Testing.RData")

faulty_training_dict = pyreadr.read_r("data/TEP_Faulty_Training.RData")
faulty_testing_dict = pyreadr.read_r("data/TEP_Faulty_Testing.RData")

DF_FF_TRAINING_RAW = fault_free_training_dict["fault_free_training"]
DF_FF_TEST_RAW = fault_free_testing_dict["fault_free_testing"]

DF_F_TRAINING_RAW = faulty_training_dict["faulty_training"]
DF_F_TEST_RAW = faulty_testing_dict["faulty_testing"]

In [ ]:
# In-control X training data
scaler_incontrol = StandardScaler()
X_INCONTROL_TRAIN_FULL_DF = DF_FF_TRAINING_RAW.drop(columns=COLUMNS_TO_REMOVE, axis=1)

scaler_incontrol.fit(X_INCONTROL_TRAIN_FULL_DF)
X_INCONTROL_TRAIN_FULL_SCALED = scaler_incontrol.transform(X_INCONTROL_TRAIN_FULL_DF)
print("In-control training data shape:", X_INCONTROL_TRAIN_FULL_SCALED.shape)

In [ ]:
simulation_run = 1
fault_number = 2

DF_FF_TRAINING = DF_FF_TRAINING_RAW.query("simulationRun == @simulation_run")
DF_FF_TEST = DF_FF_TEST_RAW.query("simulationRun == @simulation_run")

DF_F_TRAINING = DF_F_TRAINING_RAW.query("faultNumber == @fault_number and simulationRun == @simulation_run")
DF_F_TEST = DF_F_TEST_RAW.query("faultNumber == @fault_number and simulationRun == @simulation_run")


# In-control test data
#scaler_incontrol = StandardScaler()
X_INCONTROL_TRAIN_PLAY_DF = DF_FF_TRAINING.drop(columns=COLUMNS_TO_REMOVE, axis=1)
X_INCONTROL_TEST_PLAY_DF = DF_FF_TEST.drop(columns=COLUMNS_TO_REMOVE, axis=1)

scaler_incontrol.fit(X_INCONTROL_TRAIN_PLAY_DF)
X_INCONTROL_TRAIN_PLAY_SCALED = scaler_incontrol.transform(X_INCONTROL_TRAIN_PLAY_DF)
X_INCONTROL_TEST_PLAY_SCALED = scaler_incontrol.transform(X_INCONTROL_TEST_PLAY_DF)

# Out-of-control test data
X_OUT_OF_CONTROL_TEST_PLAY_DF = DF_F_TEST.drop(columns=COLUMNS_TO_REMOVE, axis=1)
X_OUT_OF_CONTROL_TRAIN_PLAY_DF = DF_F_TRAINING.drop(columns=COLUMNS_TO_REMOVE, axis=1)

X_OUT_OF_CONTROL_TEST_PLAY_SCALED = scaler_incontrol.transform(X_OUT_OF_CONTROL_TEST_PLAY_DF)
X_OUT_OF_CONTROL_TRAIN_PLAY_SCALED = scaler_incontrol.transform(X_OUT_OF_CONTROL_TRAIN_PLAY_DF)

print("In-control training data shape:", X_INCONTROL_TRAIN_PLAY_SCALED.shape)
print("In-control test data shape:", X_INCONTROL_TEST_PLAY_SCALED.shape)
print("Out-of-control training data shape:", X_OUT_OF_CONTROL_TRAIN_PLAY_SCALED.shape)
print("Out-of-control test data shape:", X_OUT_OF_CONTROL_TEST_PLAY_SCALED.shape)

## MCUSUM 


In [ ]:
# MCUSUM (MULTIVARIATE CUMULATIVE SUM) IMPLEMENTATION

from typing import Optional, Tuple
from matplotlib import pyplot as plt
import numpy as np
from numpy.typing import NDArray



class MCUSUMDetector:

    def __init__(self, k = 0.5, h = None):
        """
        Args:
            k: Reference value (sensitivity parameter)
            h: Control limit (threshold for detection)
        """
        self.k = k
        self.h = h
        self.mu_0 = None
        self.sigma = None
        self.is_fitted = False

    def fit(self, X_incontrol: NDArray[np.float64], verbose: bool = False) -> 'MCUSUMDetector':
        """
        Fit MCUSUM parameters using in-control training data.
        
        Args:
            X_incontrol: In-control (normal) training data
            
        Returns:
            Self for method chaining
        """
        if verbose:
            print(f"🔧 **Fitting MCUSUM Parameters**")

        self.mu_0, self.sigma = self._estimate_incontrol_parameters(X_incontrol)
        
        if self.h is None:
            self.h = self._estimate_control_limit(X_incontrol)
        
        self.is_fitted = True

        if verbose:
            print(f"   Mean vector shape: {self.mu_0.shape}")
            print(f"   Covariance matrix shape: {self.sigma.shape}")
            print(f"   Reference value k: {self.k:.4f}")
            print(f"   Control limit h: {self.h:.4f}")

        return self
    
    def predict(self, X_test: NDArray[np.float64]):
        if not self.is_fitted:
            raise ValueError("MCUSUM detector must be fitted before prediction")

        statistics = self._compute_mcusum_scores(X_test, self.mu_0, self.sigma, self.k)
        flags = statistics > self.h
        
        return statistics, flags
    
    @staticmethod
    def _estimate_incontrol_parameters(X_incontrol: NDArray[np.float64]) :
        """
        Estimate mean vector and covariance matrix from in-control data.
        
        Args:
            X_incontrol: In-control training data
            
        Returns:
            Tuple of (mean vector, covariance matrix)
        """
        mu_0 = np.mean(X_incontrol, axis=0)
        sigma = np.cov(X_incontrol, rowvar=False, bias=False)
        
        # Ensure positive definite covariance matrix
        min_eigenval = np.min(np.linalg.eigvals(sigma))
        if min_eigenval <= 0:
            print(f"⚠️  Warning: Adding regularization to covariance matrix (min eigenvalue: {min_eigenval:.2e})")
            sigma += np.eye(sigma.shape[0]) * abs(min_eigenval) * 1.01
        
        return mu_0, sigma
    
    @staticmethod
    def _compute_mcusum_scores(X_test: NDArray[np.float64],
                              mu_0,
                              sigma,
                              k):
        """
        Compute MCUSUM statistics for test data.
        
        Args:
            X_test: Test data
            mu_0: In-control mean vector
            sigma: In-control covariance matrix
            k: Reference value
            
        Returns:
            MCUSUM statistics for each sample
        """
        X_test = np.asarray(X_test)
        mu_0 = np.asarray(mu_0)
        sigma = np.asarray(sigma)
        
        n_samples, n_features = X_test.shape
        
        # Compute whitening transformation: Σ^{-1/2}
        try:
            eigvals, eigvecs = np.linalg.eigh(sigma)
            eigvals_inv_sqrt = np.diag(1.0 / np.sqrt(np.maximum(eigvals, 1e-12)))
            sigma_inv_sqrt = eigvecs @ eigvals_inv_sqrt @ eigvecs.T
        except np.linalg.LinAlgError:
            print("⚠️  Warning: Using pseudo-inverse for covariance matrix")
            sigma_inv_sqrt = np.linalg.pinv(sigma)
        
        # Whiten the data
        Z = (X_test - mu_0) @ sigma_inv_sqrt.T
        
        # MCUSUM recursion
        S_t = np.zeros(n_features)
        T = np.zeros(n_samples)
        
        for t in range(n_samples):
            V_t = S_t + Z[t]
            norm_V_t = np.linalg.norm(V_t)
            
            if norm_V_t <= k:
                S_t = np.zeros(n_features)
            else:
                shrinkage = 1.0 - k / norm_V_t
                S_t = V_t * shrinkage
            
            T[t] = np.linalg.norm(S_t)
        
        return T
    
    def _estimate_control_limit(self, X_incontrol: NDArray[np.float64],
                              n_simulations: int = 500,
                              percentile: float = 99.0,
                              verbose: bool = False):
        """
        Estimate control limit using Monte Carlo simulation.
        
        Args:
            X_incontrol: In-control training data
            n_simulations: Number of Monte Carlo simulations
            percentile: Percentile for control limit
            
        Returns:
            Estimated control limit
        """

        max_T_values = []
        sample_size = min(300, X_incontrol.shape[0])
        
        for i in range(n_simulations):
            # Bootstrap sample from in-control data
            indices = np.random.choice(X_incontrol.shape[0], size=sample_size, replace=True)
            sample = X_incontrol[indices]
            
            # Compute MCUSUM statistics
            T = self._compute_mcusum_scores(sample, self.mu_0, self.sigma, self.k)
            max_T_values.append(np.max(T))
        
        h = np.percentile(max_T_values, percentile)
        if verbose:
            print(f"   Control limit (h) estimated at {percentile}th percentile: {h:.4f}")
        
        return h
    
    @staticmethod
    def compute_reference_value_k(delta,
                                 sigma):
        """
        Compute optimal reference value k = 0.5 * ||Σ^{-1/2} δ||
        
        Args:
            delta: Expected shift vector
            sigma: In-control covariance matrix
            
        Returns:
            Optimal reference value
        """
        # Whitening matrix computation
        eigvals, eigvecs = np.linalg.eigh(sigma)
        eigvals_inv_sqrt = np.diag(1.0 / np.sqrt(np.maximum(eigvals, 1e-12)))
        sigma_inv_sqrt = eigvecs @ eigvals_inv_sqrt @ eigvecs.T
        
        # Transform delta and compute norm
        whitened_delta = sigma_inv_sqrt @ delta
        k = 0.5 * np.linalg.norm(whitened_delta)
        
        return k



In [ ]:
mcusum = None  # Global MCUSUM instance to be initialized later


import matplotlib.pyplot as plt
import numpy as np
from typing import Optional
from numpy.typing import NDArray


def plot_mcusum_diagnostics(
    statistics_normal: NDArray[np.float64],
    statistics_anomaly: Optional[NDArray[np.float64]] = None,
    h: float | None = 0.0,
    title_suffix: str = "",
    use_log: bool = True,
    diff_threshold: float = 1e-3
) -> None:
    """
    Create MCUSUM diagnostic plots with markers for differences and control limit crossings.
    """
    rows = 3 if statistics_anomaly is not None else 1
    fig, axs = plt.subplots(rows, 2, figsize=(15, 5 * rows))
    if rows == 1:
        axs = np.array([axs])

    # Helper to set y-axis log if requested
    def maybe_log(ax):
        if use_log:
            ax.set_yscale("log")

    # Normal histogram
    axs[0, 0].hist(statistics_normal, bins=50, alpha=0.7, color='blue', density=True)
    axs[0, 0].axvline(h, color='red', linestyle='--', linewidth=2, label=f'Control Limit (h={h:.2f})')
    maybe_log(axs[0, 0])
    axs[0, 0].set_xlabel("MCUSUM Statistic")
    axs[0, 0].set_ylabel("Density")
    axs[0, 0].set_title(f"Normal Data Histogram {title_suffix}")
    axs[0, 0].legend()
    axs[0, 0].grid(True, alpha=0.3)

    # Normal time series
    axs[0, 1].plot(statistics_normal, color='blue', alpha=0.7, label="Normal Data")
    axs[0, 1].axhline(h, color='red', linestyle='--', linewidth=2, label=f'Control Limit (h={h:.2f})')
    maybe_log(axs[0, 1])
    axs[0, 1].set_xlabel("Sample Index")
    axs[0, 1].set_ylabel("MCUSUM Statistic")
    axs[0, 1].set_title(f"Normal Data Over Time {title_suffix}")
    axs[0, 1].legend()
    axs[0, 1].grid(True, alpha=0.3)

    if statistics_anomaly is not None:
        # Anomaly histogram
        axs[1, 0].hist(statistics_anomaly, bins=50, alpha=0.7, color='red', density=True)
        axs[1, 0].axvline(h, color='red', linestyle='--', linewidth=2, label=f'Control Limit (h={h:.2f})')
        maybe_log(axs[1, 0])
        axs[1, 0].set_xlabel("MCUSUM Statistic")
        axs[1, 0].set_ylabel("Density")
        axs[1, 0].set_title(f"Anomaly Data Histogram {title_suffix}")
        axs[1, 0].legend()
        axs[1, 0].grid(True, alpha=0.3)

        # Anomaly time series
        axs[1, 1].plot(statistics_anomaly, color='red', alpha=0.7, label="Anomaly Data")
        axs[1, 1].axhline(h, color='red', linestyle='--', linewidth=2, label=f'Control Limit (h={h:.2f})')
        maybe_log(axs[1, 1])
        axs[1, 1].set_xlabel("Sample Index")
        axs[1, 1].set_ylabel("MCUSUM Statistic")
        axs[1, 1].set_title(f"Anomaly Data Over Time {title_suffix}")
        axs[1, 1].legend()
        axs[1, 1].grid(True, alpha=0.3)

        # Overlay plot
        axs[2, 0].plot(statistics_normal, color='blue', alpha=0.7, label="Normal Data")
        axs[2, 0].plot(statistics_anomaly, color='red', alpha=0.7, label="Anomaly Data")
        axs[2, 0].axhline(h, color='red', linestyle='--', linewidth=2, label=f'Control Limit (h={h:.2f})')
        maybe_log(axs[2, 0])
        axs[2, 0].set_xlabel("Sample Index")
        axs[2, 0].set_ylabel("MCUSUM Statistic")
        axs[2, 0].set_title(f"Normal vs Anomaly Overlay {title_suffix}")
        axs[2, 0].grid(True, alpha=0.3)

        # First point above threshold difference
        diff = np.abs(statistics_normal - statistics_anomaly)
        if np.any(diff > diff_threshold):
            idx_diff = np.argmax(diff > diff_threshold)
            val_norm = statistics_normal[idx_diff]
            val_anom = statistics_anomaly[idx_diff]
            axs[2, 0].axvline(idx_diff, color='green', linestyle='--', linewidth=2,
                               label=f'First Diff (idx={idx_diff})')
            # axs[2, 0].annotate(f"{val_norm:.4f} / {val_anom:.4f}",
            #                     xy=(idx_diff, max(val_norm, val_anom)),
            #                     xytext=(idx_diff + 5, max(val_norm, val_anom) * 1.05),
            #                     arrowprops=dict(arrowstyle="->", color='green'),
            #                     color='green')

        # First point above control limit
        for series, color, label_prefix in zip([statistics_normal, statistics_anomaly],
                                               ['purple', 'orange'],
                                               ['Normal', 'Anomaly']):
            if np.any(series > h):
                idx_h = np.argmax(series > h)
                val_h = series[idx_h]
                axs[2, 0].axvline(idx_h, color=color, linestyle='--', linewidth=2,
                                   label=f'{label_prefix} first above h (idx={idx_h})')
                # axs[2, 0].annotate(f"{val_h:.4f}",
                #                     xy=(idx_h, val_h),
                #                     xytext=(idx_h + 5, val_h * 1.05),
                #                     arrowprops=dict(arrowstyle="->", color=color),
                #                     color=color)

        axs[2, 0].legend(loc='upper left', bbox_to_anchor=(1.02, 1), borderaxespad=0)
        fig.delaxes(axs[2, 1])  # remove unused subplot

    plt.tight_layout()
    plt.show()


def apply_mcusum(X_INCONTROL_TEST, X_OUT_OF_CONTROL_TEST, verbose: bool = True) -> Tuple[NDArray[np.int_], NDArray[np.int_]]:
    
    stats_normal, flags_normal = mcusum.predict(X_INCONTROL_TEST)
    stats_anomaly, flags_anomaly = mcusum.predict(X_OUT_OF_CONTROL_TEST)
    
    # Create diagnostic plots
    if verbose:
        plot_mcusum_diagnostics(stats_normal, stats_anomaly, mcusum.h, "Tennessee_Eastman") # type: ignore

    # Convert boolean flags to 0 (normal) and 1 (anomaly)
    return flags_normal.astype(int), flags_anomaly.astype(int)


def mcusum_predict(x_scaled):

    stats_normal, y_pred = mcusum.predict(x_scaled)

    return y_pred.astype(int)




In [ ]:
n_features = X_INCONTROL_TRAIN_FULL_SCALED.shape[1]
delta = np.ones(n_features) * 0.1
mu_0, sigma = MCUSUMDetector._estimate_incontrol_parameters(X_INCONTROL_TRAIN_FULL_SCALED)
k = MCUSUMDetector.compute_reference_value_k(delta, sigma)
print(k)
mcusum = MCUSUMDetector(k=k ) # type: ignore
mcusum.fit(X_INCONTROL_TRAIN_FULL_SCALED)

flags_normal, flags_anomaly = apply_mcusum(
	X_INCONTROL_TEST_PLAY_SCALED,
	X_OUT_OF_CONTROL_TEST_PLAY_SCALED
)

arl0 = np.argmax(flags_normal == 1) if np.any(flags_normal == 1) else None
arl1 = np.argmax(flags_anomaly == 1) if np.any(flags_anomaly == 1) else None

print("ARL0 (In-control):", arl0)
print("ARL1 (Out-of-control):", arl1)

n = min(len(flags_normal), len(flags_anomaly))
print(np.sum(flags_normal[:n] == flags_anomaly[:n]))


**As we see in the plot "Normal vs Anomaly Overlay" the fault injection start at the point 160, so we need to cut out the data before this point for both in-control and out-of-control test sets.**

In [ ]:

n_features = X_INCONTROL_TRAIN_FULL_SCALED.shape[1]
delta = np.ones(n_features) * 0.1
mu_0, sigma = MCUSUMDetector._estimate_incontrol_parameters(X_INCONTROL_TRAIN_FULL_SCALED)
k = MCUSUMDetector.compute_reference_value_k(delta, sigma)
print(k)

mcusum = MCUSUMDetector(k=9, h=3) # type: ignore
mcusum.fit(X_INCONTROL_TRAIN_FULL_SCALED)

fault_injection_point = 160
X_INCONTROL_TEST_PLAY_SCALED_CUT = X_INCONTROL_TEST_PLAY_SCALED[fault_injection_point:]
X_OUT_OF_CONTROL_TEST_PLAY_SCALED_CUT = X_OUT_OF_CONTROL_TEST_PLAY_SCALED[fault_injection_point:]

flags_normal, flags_anomaly = apply_mcusum(
	X_INCONTROL_TEST_PLAY_SCALED_CUT,
	X_OUT_OF_CONTROL_TEST_PLAY_SCALED_CUT
)

arl0 = np.argmax(flags_normal == 1) if np.any(flags_normal == 1) else None
arl1 = np.argmax(flags_anomaly == 1) if np.any(flags_anomaly == 1) else None

print("In-control test data shape:", X_INCONTROL_TEST_PLAY_SCALED.shape)
print("Cut In-control test data shape:", X_INCONTROL_TEST_PLAY_SCALED_CUT.shape)

print("ARL0 (In-control):", arl0)
print("ARL1 (Out-of-control):", arl1)

n = min(len(flags_normal), len(flags_anomaly))
print(np.sum(flags_normal[:n] == flags_anomaly[:n]))


### Finding optimal values

In [ ]:
best_k = None
best_arl0 = -np.inf
best_arl1 = np.inf

k_values = np.linspace(0.1, 10, 30)  # Adjust range and steps as needed

for k_test in k_values:
    mcusum = MCUSUMDetector(k=k_test, h=2.8)
    mcusum.fit(X_INCONTROL_TRAIN_FULL_SCALED)
    flags_normal, flags_anomaly = apply_mcusum(
        X_INCONTROL_TEST_PLAY_SCALED_CUT,
        X_OUT_OF_CONTROL_TEST_PLAY_SCALED_CUT,
        verbose=False
    )
    arl0 = np.argmax(flags_normal == 1) if np.any(flags_normal == 1) else len(flags_normal)
    arl1 = np.argmax(flags_anomaly == 1) if np.any(flags_anomaly == 1) else len(flags_anomaly)
    
    # Prefer higher ARL0, then lower ARL1
    if (arl0 > best_arl0) or (arl0 == best_arl0 and arl1 < best_arl1):
        best_k = k_test
        best_arl0 = arl0
        best_arl1 = arl1

print(f"Best k: {best_k}, ARL0: {best_arl0}, ARL1: {best_arl1}")

# Refit with best k and show results
mcusum = MCUSUMDetector(k=best_k, h=2.8)
mcusum.fit(X_INCONTROL_TRAIN_FULL_SCALED)
flags_normal, flags_anomaly = apply_mcusum(
    X_INCONTROL_TEST_PLAY_SCALED_CUT,
    X_OUT_OF_CONTROL_TEST_PLAY_SCALED_CUT
)
arl0 = np.argmax(flags_normal == 1) if np.any(flags_normal == 1) else None
arl1 = np.argmax(flags_anomaly == 1) if np.any(flags_anomaly == 1) else None

print("Final ARL0 (In-control):", arl0)
print("Final ARL1 (Out-of-control):", arl1)


In [ ]:
best_h = None
best_arl0 = -np.inf
best_arl1 = np.inf

# Fix k to a reasonable value, e.g., from previous best_k or a constant
fixed_k = best_k  # or set to a specific value, e.g., fixed_k = 6.4

h_values = np.linspace(1.0, 10.0, 30)  # Adjust range and steps as needed

for h_test in h_values:
    mcusum = MCUSUMDetector(k=fixed_k, h=h_test)
    mcusum.fit(X_INCONTROL_TRAIN_FULL_SCALED)
    flags_normal, flags_anomaly = apply_mcusum(
        X_INCONTROL_TEST_PLAY_SCALED_CUT,
        X_OUT_OF_CONTROL_TEST_PLAY_SCALED_CUT,
        verbose=False
    )
    arl0 = np.argmax(flags_normal == 1) if np.any(flags_normal == 1) else len(flags_normal)
    arl1 = np.argmax(flags_anomaly == 1) if np.any(flags_anomaly == 1) else len(flags_anomaly)
    
    # Prefer higher ARL0, then lower ARL1
    if (arl0 > best_arl0) or (arl0 == best_arl0 and arl1 < best_arl1):
        best_h = h_test
        best_arl0 = arl0
        best_arl1 = arl1

print(f"Best h: {best_h}, ARL0: {best_arl0}, ARL1: {best_arl1}")

# Refit with best h and show results
mcusum = MCUSUMDetector(k=fixed_k, h=best_h)
mcusum.fit(X_INCONTROL_TRAIN_FULL_SCALED)
flags_normal, flags_anomaly = apply_mcusum(
    X_INCONTROL_TEST_PLAY_SCALED_CUT,
    X_OUT_OF_CONTROL_TEST_PLAY_SCALED_CUT
)
arl0 = np.argmax(flags_normal == 1) if np.any(flags_normal == 1) else None
arl1 = np.argmax(flags_anomaly == 1) if np.any(flags_anomaly == 1) else None

print("Final ARL0 (In-control):", arl0)
print("Final ARL1 (Out-of-control):", arl1)


In [ ]:
n = min(len(flags_normal), len(flags_anomaly))
print(np.sum(flags_normal[:n] == flags_anomaly[:n]))

In [ ]:
# Choose the feature index to plot (0-based)
feature_idx = 10  # Change this to select a different feature

plt.figure(figsize=(15, 6))
plt.plot(X_INCONTROL_TEST_PLAY_SCALED[:, feature_idx], label='In-Control Test Data', color='blue')
plt.plot(X_OUT_OF_CONTROL_TEST_PLAY_SCALED[:, feature_idx], label='Out-of-Control Test Data', color='red')
plt.title(f'Time Series Plot of Scaled Test Data (Feature {feature_idx})')
plt.xlabel('Time')
plt.ylabel('Scaled Value')
plt.legend()
plt.grid()
plt.show()

## Autoencoder

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

class AutoencoderDetector:
    def __init__(self, encoding_dim: int = 8):
        self.encoding_dim = encoding_dim
        self.model: models.Model | None = None
        self.threshold: float | None = None
        self.X_train: np.ndarray | None = None

    def fit(self, X_train: np.ndarray, epochs: int = 50, batch_size: int = 32, threshold_percentile: float = 95):
        self.X_train = X_train
        input_dim = X_train.shape[1]

        input_layer = layers.Input(shape=(input_dim,))
        encoded = layers.Dense(16, activation='relu')(input_layer)
        encoded = layers.Dense(self.encoding_dim, activation='relu')(encoded)
        decoded = layers.Dense(16, activation='relu')(encoded)
        decoded = layers.Dense(input_dim, activation='linear')(decoded)

        self.model = models.Model(inputs=input_layer, outputs=decoded)
        self.model.compile(optimizer='adam', loss='mse')

        self.model.fit(
            X_train,
            X_train,
            epochs=epochs,
            batch_size=batch_size,
            verbose=0,
            validation_split=0.1
        )

        # Compute threshold from training reconstruction error
        X_pred = self.model.predict(X_train)
        errors = np.mean((X_train - X_pred) ** 2, axis=1)
        self.threshold = np.percentile(errors, threshold_percentile)

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        if self.model is None or self.threshold is None or self.X_train is None:
            raise RuntimeError("Model not trained. Call fit() first.")

        X_pred = self.model.predict(X_test)
        errors = np.mean((X_test - X_pred) ** 2, axis=1)
        return (errors > self.threshold).astype(int)


In [ ]:
autoencoder_detector = AutoencoderDetector()
autoencoder_detector.fit(X_INCONTROL_TRAIN_FULL_SCALED)



In [ ]:
autoencoder_predictions_anomaly = autoencoder_detector.predict(X_OUT_OF_CONTROL_TEST_PLAY_SCALED_CUT)
autoencoder_predictions_normal = autoencoder_detector.predict(X_INCONTROL_TEST_PLAY_SCALED_CUT)

arl0 = np.argmax(autoencoder_predictions_normal == 1) if np.any(flags_normal == 1) else None
arl1 = np.argmax(autoencoder_predictions_anomaly == 1) if np.any(flags_anomaly == 1) else None

print("ARL0 (In-control):", arl0)
print("ARL1 (Out-of-control):", arl1)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, regularizers
import numpy as np
from typing import Optional, Tuple

class AutoencoderDetectorEnhanced:
    def __init__(self, encoding_dim: int = 8):
        self.encoding_dim = encoding_dim
        self.model: Optional[models.Model] = None
        self.threshold: Optional[float] = None
        self.X_train: Optional[np.ndarray] = None
        self.scaler_mean: Optional[np.ndarray] = None
        self.scaler_std: Optional[np.ndarray] = None
        self.history = None
        
    def _normalize(self, X: np.ndarray, fit: bool = False) -> np.ndarray:
        """Normalize data using z-score normalization."""
        if fit:
            self.scaler_mean = np.mean(X, axis=0)
            self.scaler_std = np.std(X, axis=0) + 1e-7  # Add small epsilon to avoid division by zero
        
        if self.scaler_mean is None or self.scaler_std is None:
            raise RuntimeError("Scaler not fitted. This should not happen.")
            
        return (X - self.scaler_mean) / self.scaler_std
    
    def _build_model(self, input_dim: int) -> models.Model:
        """Build an improved autoencoder architecture."""
        # Calculate layer dimensions dynamically
        layer1_dim = max(input_dim // 2, self.encoding_dim * 4)
        layer2_dim = max(input_dim // 4, self.encoding_dim * 2)
        
        # Encoder
        input_layer = layers.Input(shape=(input_dim,))
        
        # Add noise for denoising autoencoder capability (helps with robustness)
        noisy_input = layers.GaussianNoise(0.1)(input_layer)
        
        # Encoder layers with batch normalization and dropout for regularization
        encoded = layers.Dense(
            layer1_dim, 
            activation='relu',
            kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
            kernel_initializer='he_normal'
        )(noisy_input)
        encoded = layers.BatchNormalization()(encoded)
        encoded = layers.Dropout(0.2)(encoded)
        
        encoded = layers.Dense(
            layer2_dim, 
            activation='relu',
            kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
            kernel_initializer='he_normal'
        )(encoded)
        encoded = layers.BatchNormalization()(encoded)
        encoded = layers.Dropout(0.2)(encoded)
        
        # Bottleneck layer
        encoded = layers.Dense(
            self.encoding_dim, 
            activation='relu',
            kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
            kernel_initializer='he_normal',
            name='bottleneck'
        )(encoded)
        
        # Decoder layers (symmetric to encoder)
        decoded = layers.Dense(
            layer2_dim, 
            activation='relu',
            kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
            kernel_initializer='he_normal'
        )(encoded)
        decoded = layers.BatchNormalization()(decoded)
        decoded = layers.Dropout(0.2)(decoded)
        
        decoded = layers.Dense(
            layer1_dim, 
            activation='relu',
            kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
            kernel_initializer='he_normal'
        )(decoded)
        decoded = layers.BatchNormalization()(decoded)
        decoded = layers.Dropout(0.2)(decoded)
        
        # Output layer with tanh activation (works better with normalized data)
        decoded = layers.Dense(
            input_dim, 
            activation='tanh',
            kernel_initializer='glorot_uniform'
        )(decoded)
        
        model = models.Model(inputs=input_layer, outputs=decoded)
        return model

    def fit(self, X_train: np.ndarray, epochs: int = 50, batch_size: int = 32, 
            threshold_percentile: float = 95) -> None:
        """
        Train the autoencoder on normal data.
        
        Args:
            X_train: Training data (assumed to be mostly normal samples)
            epochs: Number of training epochs
            batch_size: Batch size for training
            threshold_percentile: Percentile for threshold calculation
        """
        # Normalize the training data
        X_train_normalized = self._normalize(X_train, fit=True)
        self.X_train = X_train_normalized
        
        input_dim = X_train.shape[1]
        
        # Build the model
        self.model = self._build_model(input_dim)
        
        # Use a custom learning rate schedule
        initial_learning_rate = 0.001
        lr_schedule = callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=0
        )
        
        # Early stopping to prevent overfitting
        early_stopping = callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=0
        )
        
        # Compile with Adam optimizer with custom learning rate
        optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)
        self.model.compile(
            optimizer=optimizer, 
            loss='mse',
            metrics=['mae']  # Add MAE as additional metric
        )
        
        # Train the model
        self.history = self.model.fit(
            X_train_normalized,
            X_train_normalized,
            epochs=epochs,
            batch_size=batch_size,
            verbose=0,
            validation_split=0.15,  # Slightly larger validation split
            callbacks=[lr_schedule, early_stopping],
            shuffle=True
        )
        
        # Compute reconstruction errors for threshold calculation
        X_pred = self.model.predict(X_train_normalized, verbose=0)
        
        # Calculate reconstruction error using multiple metrics
        mse_errors = np.mean((X_train_normalized - X_pred) ** 2, axis=1)
        mae_errors = np.mean(np.abs(X_train_normalized - X_pred), axis=1)
        
        # Combine errors (weighted average of MSE and MAE)
        combined_errors = 0.7 * mse_errors + 0.3 * mae_errors
        
        # Set threshold using the specified percentile
        self.threshold = np.percentile(combined_errors, threshold_percentile)
        
        # Store some statistics for potential debugging
        self.error_stats = {
            'mean': np.mean(combined_errors),
            'std': np.std(combined_errors),
            'min': np.min(combined_errors),
            'max': np.max(combined_errors),
            'threshold': self.threshold
        }

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        """
        Predict anomalies in test data.
        
        Args:
            X_test: Test data to check for anomalies
            
        Returns:
            Binary array where 1 indicates anomaly, 0 indicates normal
        """
        if self.model is None or self.threshold is None or self.X_train is None:
            raise RuntimeError("Model not trained. Call fit() first.")
        
        # Normalize test data using training statistics
        X_test_normalized = self._normalize(X_test, fit=False)
        
        # Get predictions
        X_pred = self.model.predict(X_test_normalized, verbose=0)
        
        # Calculate reconstruction errors (same combination as in training)
        mse_errors = np.mean((X_test_normalized - X_pred) ** 2, axis=1)
        mae_errors = np.mean(np.abs(X_test_normalized - X_pred), axis=1)
        combined_errors = 0.7 * mse_errors + 0.3 * mae_errors
        
        # Return binary predictions
        return (combined_errors > self.threshold).astype(int)
    
    def get_anomaly_scores(self, X_test: np.ndarray) -> np.ndarray:
        """
        Get anomaly scores (reconstruction errors) for test data.
        
        Args:
            X_test: Test data
            
        Returns:
            Array of anomaly scores (higher = more anomalous)
        """
        if self.model is None or self.X_train is None:
            raise RuntimeError("Model not trained. Call fit() first.")
        
        # Normalize test data
        X_test_normalized = self._normalize(X_test, fit=False)
        
        # Get predictions
        X_pred = self.model.predict(X_test_normalized, verbose=0)
        
        # Calculate and return reconstruction errors
        mse_errors = np.mean((X_test_normalized - X_pred) ** 2, axis=1)
        mae_errors = np.mean(np.abs(X_test_normalized - X_pred), axis=1)
        combined_errors = 0.7 * mse_errors + 0.3 * mae_errors
        
        return combined_errors
    
    def get_encoder_output(self, X: np.ndarray) -> np.ndarray:
        """
        Get the encoded representation of input data.
        
        Args:
            X: Input data
            
        Returns:
            Encoded representation from the bottleneck layer
        """
        if self.model is None:
            raise RuntimeError("Model not trained. Call fit() first.")
        
        # Create encoder model
        encoder = models.Model(
            inputs=self.model.input,
            outputs=self.model.get_layer('bottleneck').output
        )
        
        # Normalize and encode
        X_normalized = self._normalize(X, fit=False)
        return encoder.predict(X_normalized, verbose=0)

In [ ]:
autoencoder_detector_enhanced = AutoencoderDetectorEnhanced()
autoencoder_detector_enhanced.fit(X_INCONTROL_TRAIN_FULL_SCALED)

In [ ]:
autoencoder_predictions_anomaly = autoencoder_detector_enhanced.predict(X_OUT_OF_CONTROL_TEST_PLAY_SCALED_CUT)
autoencoder_predictions_normal = autoencoder_detector_enhanced.predict(X_INCONTROL_TEST_PLAY_SCALED_CUT)

arl0 = np.argmax(autoencoder_predictions_normal == 1) if np.any(flags_normal == 1) else None
arl1 = np.argmax(autoencoder_predictions_anomaly == 1) if np.any(flags_anomaly == 1) else None

print("ARL0 (In-control):", arl0)
print("ARL1 (Out-of-control):", arl1)

## EWMA

In [ ]:
import pandas as pd
import numpy as np
from scipy.linalg import eigh
from typing import Dict, Any, List

# Define classes for MEWMA variants

import numpy as np

class BaseEWMA:
    """
    Base class for EWMA-type multivariate control charts.
    Handles training mean and covariance estimation,
    and provides attributes used by specific MEWMA variants.
    """

    def __init__(self, lambda_: float = 0.2, percentile: float = 99.5):
        """
        Parameters
        ----------
        lambda_ : float
            Smoothing parameter for EWMA (0 < lambda_ <= 1).
            Higher values react faster to changes.
        percentile : float
            Percentile of in-control distribution used to set control limit.
        """
        self.lambda_ = lambda_
        self.percentile = percentile
        self.mu0 = None           # In-control mean vector
        self.p = None             # Number of variables
        self.Sigma = None         # In-control covariance matrix
        self.Sigma_inv = None     # Inverse covariance
        self.h = None             # Control limit threshold

    def fit(self, X_train: np.ndarray) -> None:
        """
        Estimate in-control parameters from training data.

        Parameters
        ----------
        X_train : np.ndarray of shape (n_samples, n_features)
            In-control training observations.
        """
        self.mu0 = np.mean(X_train, axis=0)
        self.p = X_train.shape[1]
        self.Sigma = np.cov(X_train.T)
        self.Sigma_inv = np.linalg.inv(self.Sigma)




### MEWMA

In [ ]:
class StandardMEWMA(BaseEWMA):
    """
    Standard Multivariate EWMA (MEWMA) Control Chart.

    Monitors the smoothed mean vector of a multivariate process.
    Detection statistic is a Hotelling-type T² on the EWMA vector.
    """

    def fit(self, X_train: np.ndarray) -> None:
        """
        Train the MEWMA chart on in-control data.
        Estimates the empirical distribution of the monitoring statistic
        and sets the control limit threshold.

        Parameters
        ----------
        X_train : np.ndarray of shape (n_samples, n_features)
            In-control training data.
        """
        # Step 1: Fit base parameters (mean, covariance, inverse covariance)
        super().fit(X_train)

        # Step 2: Initialize smoothed deviation vector Z
        n = X_train.shape[0]
        Z = np.zeros(self.p)
        Ts = []  # store monitoring statistics

        # Step 3: Iterate through training samples
        for i in range(n):
            # Deviation from in-control mean
            g = X_train[i] - self.mu0

            # Update EWMA vector
            Z = self.lambda_ * g + (1 - self.lambda_) * Z

            # Step 4: Monitoring statistic
            # Hotelling-type quadratic form, scaled by factor (2 - λ)/λ
            T = ((2 - self.lambda_) / self.lambda_) * np.dot(Z, self.Sigma_inv @ Z)
            Ts.append(T)

        # Step 5: Set control limit from empirical distribution
        self.h = np.percentile(Ts, self.percentile) if Ts else np.inf

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Apply the trained MEWMA chart to new observations.

        Parameters
        ----------
        X : np.ndarray of shape (n_samples, n_features)
            Test sequence of observations.

        Returns
        -------
        pred : np.ndarray of shape (n_samples,)
            Boolean array: True if alarm (out-of-control), False otherwise.
        """
        n = X.shape[0]
        Z = np.zeros(self.p)
        pred = np.zeros(n, dtype=bool)

        for i in range(n):
            # Deviation from in-control mean
            g = X[i] - self.mu0

            # Update EWMA vector
            Z = self.lambda_ * g + (1 - self.lambda_) * Z

            # Compute monitoring statistic
            T = ((2 - self.lambda_) / self.lambda_) * np.dot(Z, self.Sigma_inv @ Z)

            # Compare against control limit
            pred[i] = T > self.h

        return pred


In [ ]:
mewma = StandardMEWMA()
mewma.fit(X_INCONTROL_TRAIN_FULL_SCALED)
mewma_predictions_normal = mewma.predict(X_INCONTROL_TEST_PLAY_SCALED_CUT)
mewma_predictions_anomaly = mewma.predict(X_OUT_OF_CONTROL_TEST_PLAY_SCALED_CUT)

arl0 = np.argmax(mewma_predictions_normal == 1) if np.any(flags_normal == 1) else None
arl1 = np.argmax(mewma_predictions_anomaly == 1) if np.any(flags_anomaly == 1) else None

print("ARL0 (In-control):", arl0)
print("ARL1 (Out-of-control):", arl1)

### MEWMS

In [ ]:
class MEWMS(BaseEWMA):
    """
    Multivariate Exponentially Weighted Moving Covariance (MEWMS) Chart.

    Monitors changes in the covariance structure by smoothing
    the outer product of deviations, standardized by the in-control covariance.
    """

    def fit(self, X_train: np.ndarray) -> None:
        """
        Train the MEWMS chart on in-control data.
        Estimates the distribution of the monitoring statistic
        and sets the control limit threshold.

        Parameters
        ----------
        X_train : np.ndarray
            In-control training data.
        """
        # Step 1: Fit base parameters (mean, covariance, inverse covariance)
        super().fit(X_train)

        # Step 2: Initialize smoothed covariance statistic Y
        # Y is a weighted moving average of outer products of deviations
        n = X_train.shape[0]
        Y = np.zeros((self.p, self.p))
        Ts = []  # store monitoring statistics for threshold estimation

        # Step 3: Iterate through training samples
        for i in range(n):
            # Deviation from in-control mean
            g = X_train[i] - self.mu0

            # Update smoothed covariance estimate (EWMA form)
            Y = self.lambda_ * np.outer(g, g) + (1 - self.lambda_) * Y

            # Step 4: Monitoring statistic (standardized quadratic form)
            # Using trace(Sigma_inv @ Y) normalizes by in-control variance/correlation
            T = np.trace(self.Sigma_inv @ Y)
            Ts.append(T)

        # Step 5: Set control limit from empirical distribution
        # Use high percentile of in-control statistic as threshold
        self.h = np.percentile(Ts, self.percentile) if Ts else np.inf

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Apply the trained MEWMS chart to new observations.

        Parameters
        ----------
        X : np.ndarray of shape (n_samples, n_features)
            Test sequence of observations.

        Returns
        -------
        pred : np.ndarray of shape (n_samples,)
            Boolean array: True if alarm (out-of-control), False otherwise.
        """
        n = X.shape[0]
        Y = np.zeros((self.p, self.p))
        pred = np.zeros(n, dtype=bool)

        for i in range(n):
            # Deviation from in-control mean
            g = X[i] - self.mu0

            # Update smoothed covariance statistic
            Y = self.lambda_ * np.outer(g, g) + (1 - self.lambda_) * Y

            # Compute standardized monitoring statistic
            T = np.trace(self.Sigma_inv @ Y)

            # Compare against control limit
            pred[i] = T > self.h

        return pred.astype(int)


In [ ]:
mewms = MEWMS()
mewms.fit(X_INCONTROL_TRAIN_FULL_SCALED)
mewms_predictions_normal = mewms.predict(X_INCONTROL_TEST_PLAY_SCALED_CUT)
mewms_predictions_anomaly = mewms.predict(X_OUT_OF_CONTROL_TEST_PLAY_SCALED_CUT)

arl0 = np.argmax(mewms_predictions_normal == 1) if np.any(flags_normal == 1) else None
arl1 = np.argmax(mewms_predictions_anomaly == 1) if np.any(flags_anomaly == 1) else None

print("ARL0 (In-control):", arl0)
print("ARL1 (Out-of-control):", arl1)

### MEWMV

In [ ]:
import numpy as np

class MEWMV(BaseEWMA):
    """
    Multivariate EWMA for Mean and Variance (MEWMV) Control Chart.

    Monitors both:
    - Mean shifts (via smoothed EWMA vector Z)
    - Variance/covariance changes (via smoothed covariance matrix S)

    Detection statistic combines the standardized mean part (Hotelling-type)
    and the variance part (scaled covariance trace).
    """

    def fit(self, X_train: np.ndarray) -> None:
        """
        Train the MEWMV chart on in-control data.
        Estimates the empirical distribution of the monitoring statistic
        and sets the control limit threshold.

        Parameters
        ----------
        X_train : np.ndarray of shape (n_samples, n_features)
            In-control training data.
        """
        # Step 1: Fit base parameters (mean, covariance, inverse covariance)
        super().fit(X_train)

        # Step 2: Initialize smoothed mean and variance statistics
        n = X_train.shape[0]
        Z = np.zeros(self.p)               # EWMA mean shift vector
        S = np.zeros((self.p, self.p))     # EWMA covariance estimate
        Ts = []

        # Step 3: Iterate through training samples
        for i in range(n):
            # Mean deviation
            g_mean = X_train[i] - self.mu0

            # Update smoothed mean shift (EWMA form)
            Z = self.lambda_ * g_mean + (1 - self.lambda_) * Z

            # Update smoothed covariance contribution
            g_var = np.outer(g_mean, g_mean)
            S = self.lambda_ * g_var + (1 - self.lambda_) * S

            # Step 4: Monitoring statistic
            # Mean part: standardized quadratic form
            T_mean = ((2 - self.lambda_) / self.lambda_) * np.dot(Z, self.Sigma_inv @ Z)

            # Variance part: standardized covariance trace
            # Using trace(Sigma_inv @ S) instead of raw trace(S)
            T_var = np.trace(self.Sigma_inv @ S)

            # Combined statistic
            T = T_mean + T_var
            Ts.append(T)

        # Step 5: Set control limit from empirical in-control distribution
        self.h = np.percentile(Ts, self.percentile) if Ts else np.inf

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Apply the trained MEWMV chart to new observations.

        Parameters
        ----------
        X : np.ndarray of shape (n_samples, n_features)
            Test sequence of observations.

        Returns
        -------
        pred : np.ndarray of shape (n_samples,)
            Boolean array: True if alarm (out-of-control), False otherwise.
        """
        n = X.shape[0]
        Z = np.zeros(self.p)
        S = np.zeros((self.p, self.p))
        pred = np.zeros(n, dtype=bool)

        for i in range(n):
            # Mean deviation
            g_mean = X[i] - self.mu0

            # Update smoothed mean
            Z = self.lambda_ * g_mean + (1 - self.lambda_) * Z

            # Update smoothed covariance
            g_var = np.outer(g_mean, g_mean)
            S = self.lambda_ * g_var + (1 - self.lambda_) * S

            # Monitoring statistic
            T_mean = ((2 - self.lambda_) / self.lambda_) * np.dot(Z, self.Sigma_inv @ Z)
            T_var = np.trace(self.Sigma_inv @ S)
            T = T_mean + T_var

            # Compare against control limit
            pred[i] = T > self.h

        return pred.astype(int)


In [ ]:
mewmv = MEWMV()
mewmv.fit(X_INCONTROL_TRAIN_FULL_SCALED)
mewmv_predictions_normal = mewmv.predict(X_INCONTROL_TEST_PLAY_SCALED_CUT)
mewmv_predictions_anomaly = mewmv.predict(X_OUT_OF_CONTROL_TEST_PLAY_SCALED_CUT)

arl0 = np.argmax(mewmv_predictions_normal == 1) if np.any(flags_normal == 1) else None
arl1 = np.argmax(mewmv_predictions_anomaly == 1) if np.any(flags_anomaly == 1) else None

print("ARL0 (In-control):", arl0)
print("ARL1 (Out-of-control):", arl1)

### REWMV

In [ ]:
class REWMV(BaseEWMA):
    """
    Robust EWMA for Monitoring Variance (REWMV).

    Focuses on detecting variance shifts in multivariate processes.
    Uses log-squared deviations to reduce the impact of outliers,
    then applies EWMA smoothing and sums across dimensions.
    """

    def fit(self, X_train: np.ndarray) -> None:
        """
        Train the REWMV chart on in-control data.
        Estimates the empirical distribution of the monitoring statistic
        and sets the control limit threshold.

        Parameters
        ----------
        X_train : np.ndarray of shape (n_samples, n_features)
            In-control training data.
        """
        # Step 1: Fit base parameters (mean, covariance, inverse covariance)
        super().fit(X_train)

        # Step 2: Initialize smoothed log-variance vector
        n = X_train.shape[0]
        Y = np.zeros(self.p)   # EWMA of log-squared deviations
        Ts = []

        # Step 3: Iterate through training samples
        for i in range(n):
            # Squared deviation from mean
            diffs = (X_train[i] - self.mu0) ** 2

            # Log-transform (robust variance signal)
            g = np.log(diffs + 1e-10)  # Add small constant to avoid log(0)

            # Update smoothed statistic
            Y = self.lambda_ * g + (1 - self.lambda_) * Y

            # Step 4: Monitoring statistic
            # Sum across dimensions → global variance change measure
            T = np.sum(Y)
            Ts.append(T)

        # Step 5: Control limit
        self.h = np.percentile(Ts, self.percentile) if Ts else np.inf

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Apply the trained REWMV chart to new observations.

        Parameters
        ----------
        X : np.ndarray of shape (n_samples, n_features)
            Test sequence of observations.

        Returns
        -------
        pred : np.ndarray of shape (n_samples,)
            Boolean array: True if alarm (out-of-control), False otherwise.
        """
        n = X.shape[0]
        Y = np.zeros(self.p)
        pred = np.zeros(n, dtype=bool)

        for i in range(n):
            # Squared deviation
            diffs = (X[i] - self.mu0) ** 2

            # Log transform
            g = np.log(diffs + 1e-10)

            # Update smoothed statistic
            Y = self.lambda_ * g + (1 - self.lambda_) * Y

            # Monitoring statistic
            T = np.sum(Y)

            # Compare to control limit
            pred[i] = T > self.h

        return pred.astype(int)


In [ ]:
rewmv = REWMV()
rewmv.fit(X_INCONTROL_TRAIN_FULL_SCALED)
rewmv_predictions_normal = rewmv.predict(X_INCONTROL_TEST_PLAY_SCALED_CUT)
rewmv_predictions_anomaly = rewmv.predict(X_OUT_OF_CONTROL_TEST_PLAY_SCALED_CUT)

arl0 = np.argmax(rewmv_predictions_normal == 1) if np.any(flags_normal == 1) else None
arl1 = np.argmax(rewmv_predictions_anomaly == 1) if np.any(flags_anomaly == 1) else None

print("ARL0 (In-control):", arl0)
print("ARL1 (Out-of-control):", arl1)

### MAX MEWMA

In [ ]:
class MaxMEWMV:
    """
    Maximum Multivariate Exponentially Weighted Moving Variance (MaxMEWMV) Control Chart.

    This method extends the concept of MEWMA (Multivariate EWMA) by monitoring
    the variance structure of the process over time. Instead of only tracking
    shifts in the mean vector, it is also sensitive to covariance changes.

    Output: Binary classification
        - 0 → In-control (normal)
        - 1 → Out-of-control (anomaly)
    """

    def __init__(self, lambda_: float = 0.2, threshold: float = 10.0) -> None:
        """
        Initialize the MaxMEWMV chart.

        Parameters
        ----------
        lambda_ : float, default=0.2
            Smoothing parameter (0 < lambda_ ≤ 1).
            Small values → long memory (slower adaptation).
            Large values → short memory (faster adaptation).
        threshold : float, default=10.0
            Control limit threshold. Values above this
            are classified as anomalies.
        """
        self.lambda_: float = lambda_
        self.threshold: float = threshold
        self.mu0: Optional[np.ndarray] = None
        self.cov0: Optional[np.ndarray] = None
        self.stat_history: list[float] = []

    def fit(self, X_train: np.ndarray) -> None:
        """
        Fit the MaxMEWMV chart using in-control training data.

        Parameters
        ----------
        X_train : np.ndarray
            2D array of shape (n_samples, n_features).
            Assumed to represent only in-control (normal) observations.
        """
        # Ensure input is 2D
        if X_train.ndim != 2:
            raise ValueError("X_train must be a 2D array.")

        # Store baseline mean and covariance (in-control reference)
        self.mu0 = np.mean(X_train, axis=0)
        self.cov0 = np.cov(X_train, rowvar=False)

        # Initialize monitoring statistic with zero
        n_features: int = X_train.shape[1]
        self.stat_history = [0.0] * n_features

        # Iterate over each sample in training to initialize control structure
        for i in range(X_train.shape[0]):
            deviation: np.ndarray = X_train[i] - self.mu0
            g: np.ndarray = np.outer(deviation, deviation)

            # Update exponentially weighted moving variance statistics
            self.stat_history = (
                self.lambda_ * np.diag(g) + (1 - self.lambda_) * np.array(self.stat_history)
            ).tolist()

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Predict anomalies in new data.

        Parameters
        ----------
        X : np.ndarray
            2D array of shape (n_samples, n_features).
            Test or monitoring data (may include anomalies).

        Returns
        -------
        np.ndarray
            1D array of shape (n_samples,) with binary predictions:
            - 0 → Normal (in-control)
            - 1 → Anomaly (out-of-control)
        """
        if self.mu0 is None or self.cov0 is None:
            raise RuntimeError("Model must be fitted before prediction.")

        n_samples, n_features = X.shape
        predictions: list[int] = []

        # Initialize local monitoring statistic
        stat_local: list[float] = [0.0] * n_features

        for i in range(n_samples):
            deviation: np.ndarray = X[i] - self.mu0
            g: np.ndarray = np.outer(deviation, deviation)

            # Update variance statistics
            stat_local = (
                self.lambda_ * np.diag(g) + (1 - self.lambda_) * np.array(stat_local)
            ).tolist()

            # Use maximum variance component as monitoring statistic
            stat_value: float = float(np.max(stat_local))

            # Apply threshold to generate binary classification
            if stat_value > self.threshold:
                predictions.append(1)  # anomaly
            else:
                predictions.append(0)  # normal

        return np.array(predictions)


In [ ]:
max_mewmv = MaxMEWMV()
max_mewmv.fit(X_INCONTROL_TRAIN_FULL_SCALED)
max_mewmv_predictions_normal = max_mewmv.predict(X_INCONTROL_TEST_PLAY_SCALED_CUT)
max_mewmv_predictions_anomaly = max_mewmv.predict(X_OUT_OF_CONTROL_TEST_PLAY_SCALED_CUT)

arl0 = np.argmax(max_mewmv_predictions_normal == 1) if np.any(flags_normal == 1) else None
arl1 = np.argmax(max_mewmv_predictions_anomaly == 1) if np.any(flags_anomaly == 1) else None

print("ARL0 (In-control):", arl0)
print("ARL1 (Out-of-control):", arl1)

### MNSE

In [ ]:
class MNSE(BaseEWMA):
    def fit(self, X_train: np.ndarray):
        super().fit(X_train)
        n = X_train.shape[0]
        Y = np.zeros(self.p)
        Ts = []
        for i in range(n):
            diff = X_train[i] - self.mu0
            norm = np.linalg.norm(diff)
            g = diff / norm if norm > 1e-10 else np.zeros(self.p)
            Y = self.lambda_ * g + (1 - self.lambda_) * Y
            T = np.dot(Y, Y)
            Ts.append(T)
        self.h = np.percentile(Ts, self.percentile) if Ts else np.inf

    def predict(self, X: np.ndarray) -> np.ndarray:
        n = X.shape[0]
        Y = np.zeros(self.p)
        pred = np.zeros(n, dtype=bool)
        for i in range(n):
            diff = X[i] - self.mu0
            norm = np.linalg.norm(diff)
            g = diff / norm if norm > 1e-10 else np.zeros(self.p)
            Y = self.lambda_ * g + (1 - self.lambda_) * Y
            T = np.dot(Y, Y)
            pred[i] = T > self.h
        return pred.astype(int)

# For adaptive variants, providing placeholders as they require additional logic

class AEWMA(StandardMEWMA):
    # Placeholder for AEWMA: lambda varies based on recent T_t
    # For simplicity, uses fixed lambda; extend with adaptive logic if needed
    pass

class SAMEWMA(StandardMEWMA):
    # Placeholder for SAMEWMA: integrate with external ML model
    # Example: def set_lambda(self, ml_model_prediction): ...
    pass

class AMFEWMA(StandardMEWMA):
    # Placeholder for AMFEWMA: similar to standard for multivariate; extend for basis coefficients if data is functional
    pass

In [ ]:
mnse = MNSE()
mnse.fit(X_INCONTROL_TRAIN_FULL_SCALED)
mnse_predictions_normal = mnse.predict(X_INCONTROL_TEST_PLAY_SCALED_CUT)
mnse_predictions_anomaly = mnse.predict(X_OUT_OF_CONTROL_TEST_PLAY_SCALED_CUT)

arl0 = np.argmax(mnse_predictions_normal == 1) if np.any(flags_normal == 1) else None
arl1 = np.argmax(mnse_predictions_anomaly == 1) if np.any(flags_anomaly == 1) else None

print("ARL0 (In-control):", arl0)
print("ARL1 (Out-of-control):", arl1)

## Run with all samples

In [ ]:
import numpy as np

def get_first_false_alarm_index(flags_normal):
    if np.any(flags_normal == 1):
        return int(np.argmax(flags_normal == 1))
    return None

def get_first_detection_delay(flags_anomaly):
    if np.any(flags_anomaly == 1):
        return int(np.argmax(flags_anomaly == 1))
    return None

In [ ]:
SIMULATION_RUN_RANGE = range(1, 21)
FAULT_NUMBER_RANGE = range(1, 21)

results: List[Dict[str, Any]] = []

# Add to MODELS dictionary

# Dictionary of models for extensibility
MODELS = {
    "MCUSUM": mcusum_predict,
    "Autoencoder": autoencoder_detector.predict,
    "AutoencoderEnhanced": autoencoder_detector_enhanced.predict,
    # "MEWMS": mewms.predict
}

for simulation_run in SIMULATION_RUN_RANGE:
    DF_FF_TRAINING_SEQUENCED = DF_FF_TRAINING_RAW.query("simulationRun == @simulation_run")
    DF_FF_TEST_SEQUENCED = DF_FF_TEST_RAW.query("simulationRun == @simulation_run")

    # In-control data
    X_INCONTROL_TRAIN_DF = DF_FF_TRAINING_SEQUENCED.drop(columns=COLUMNS_TO_REMOVE, axis=1)
    X_INCONTROL_TEST_DF = DF_FF_TEST_SEQUENCED.drop(columns=COLUMNS_TO_REMOVE, axis=1)

    scaler_incontrol.fit(X_INCONTROL_TRAIN_DF)
    X_INCONTROL_TRAIN_SCALED = scaler_incontrol.transform(X_INCONTROL_TRAIN_DF)[fault_injection_point:]
    X_INCONTROL_TEST_SCALED = scaler_incontrol.transform(X_INCONTROL_TEST_DF)[fault_injection_point:]

    # Precompute in-control predictions once per simulation run
    pred_normal_dict = {}
    for model_name, model_func in MODELS.items():
        pred_normal_dict[model_name] = model_func(X_INCONTROL_TEST_SCALED)

    for fault_number in FAULT_NUMBER_RANGE:
        DF_F_TRAINING_SEQUENCED = DF_F_TRAINING_RAW.query(
            "faultNumber == @fault_number and simulationRun == @simulation_run"
        )
        DF_F_TEST_SEQUENCED = DF_F_TEST_RAW.query(
            "faultNumber == @fault_number and simulationRun == @simulation_run"
        )

        # Out-of-control data
        X_OUT_OF_CONTROL_TEST_DF = DF_F_TEST_SEQUENCED.drop(columns=COLUMNS_TO_REMOVE, axis=1)
        X_OUT_OF_CONTROL_TRAIN_DF = DF_F_TRAINING_SEQUENCED.drop(columns=COLUMNS_TO_REMOVE, axis=1)

        X_OUT_OF_CONTROL_TEST_SCALED = scaler_incontrol.transform(X_OUT_OF_CONTROL_TEST_DF)[fault_injection_point:]
        X_OUT_OF_CONTROL_TRAIN_SCALED = scaler_incontrol.transform(X_OUT_OF_CONTROL_TRAIN_DF)[fault_injection_point:]

        # Run all models
        for model_name, model_func in MODELS.items():
            pred_anomaly = model_func(X_OUT_OF_CONTROL_TEST_SCALED)
            pred_normal = pred_normal_dict[model_name]

            arl1 = get_first_detection_delay(pred_anomaly)
            arl0 = get_first_false_alarm_index(pred_normal)

            print(f"**Simulation Run: {simulation_run}, Fault Number: {fault_number}, Model: {model_name}**")
            print(f"ARL0 (False Alarm): {arl0} ARL1 (Detection Delay): {arl1}")
            print()

            detection_fraction = np.mean(pred_anomaly)

            results.append({
                "simulationRun": simulation_run,
                "faultNumber": fault_number,
                "model": model_name,
                "ARL0": arl0,
                "ARL1": arl1,
                "detection_fraction": detection_fraction
            })

# Convert results to DataFrame and aggregate metrics (unchanged)
df_results = pd.DataFrame(results)

grouped = df_results.groupby(['model', 'faultNumber'])
conditional_arl0 = grouped['ARL0'].apply(lambda x: x.dropna().mean() if not x.dropna().empty else np.nan)
sdrl0 = grouped['ARL0'].apply(lambda x: x.dropna().std() if len(x.dropna()) > 1 else np.nan)
non_fa_fraction = grouped['ARL0'].apply(lambda x: x.isnull().mean())
conditional_arl1 = grouped['ARL1'].apply(lambda x: x.dropna().mean() if not x.dropna().empty else np.nan)
sdrl1 = grouped['ARL1'].apply(lambda x: x.dropna().std() if len(x.dropna()) > 1 else np.nan)
non_detection_fraction = grouped['ARL1'].apply(lambda x: x.isnull().mean())
avg_detection_fraction = grouped['detection_fraction'].mean()

summary_df = pd.DataFrame({
    'conditional_ARL0': conditional_arl0,
    'SDRL0': sdrl0,
    'non_FA_fraction': non_fa_fraction,
    'conditional_ARL1': conditional_arl1,
    'SDRL1': sdrl1,
    'non_detection_fraction': non_detection_fraction,
    'avg_detection_fraction': avg_detection_fraction
}).reset_index()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Any, Tuple
import warnings
warnings.filterwarnings('ignore')

class ModelComparisonAnalyzer:
    """Class to analyze and visualize model comparison results."""
    
    def __init__(self, df_results: pd.DataFrame, summary_df: pd.DataFrame):
        """
        Initialize the analyzer with results data.
        
        Args:
            df_results: DataFrame with raw results from all simulation runs
            summary_df: DataFrame with aggregated summary statistics
        """
        self.df_results = df_results
        self.summary_df = summary_df
        self.models = df_results['model'].unique()
        self.fault_numbers = sorted(df_results['faultNumber'].unique())
        
    def create_comparison_table(self, metric: str = 'all') -> pd.DataFrame:
        """
        Create a detailed comparison table for specified metrics.
        
        Args:
            metric: 'ARL0', 'ARL1', 'detection', or 'all'
        
        Returns:
            Formatted comparison DataFrame
        """
        if metric == 'all':
            # Create comprehensive comparison table
            pivot_tables = []
            
            # ARL1 comparison
            arl1_pivot = self.summary_df.pivot(
                index='faultNumber', 
                columns='model', 
                values='conditional_ARL1'
            ).round(2)
            arl1_pivot.columns = [f'{col}_ARL1' for col in arl1_pivot.columns]
            pivot_tables.append(arl1_pivot)
            
            # ARL0 comparison
            arl0_pivot = self.summary_df.pivot(
                index='faultNumber', 
                columns='model', 
                values='conditional_ARL0'
            ).round(2)
            arl0_pivot.columns = [f'{col}_ARL0' for col in arl0_pivot.columns]
            pivot_tables.append(arl0_pivot)
            
            # Detection fraction comparison
            det_pivot = self.summary_df.pivot(
                index='faultNumber', 
                columns='model', 
                values='avg_detection_fraction'
            ).round(3)
            det_pivot.columns = [f'{col}_DetFrac' for col in det_pivot.columns]
            pivot_tables.append(det_pivot)
            
            # Combine all metrics
            comparison_table = pd.concat(pivot_tables, axis=1)
            comparison_table.index.name = 'Fault'
            
            return comparison_table
        
        elif metric == 'ARL1':
            return self._create_arl1_table()
        elif metric == 'ARL0':
            return self._create_arl0_table()
        elif metric == 'detection':
            return self._create_detection_table()
        else:
            raise ValueError("metric must be 'ARL0', 'ARL1', 'detection', or 'all'")
    
    def _create_arl1_table(self) -> pd.DataFrame:
        """Create ARL1 comparison table with mean and std."""
        arl1_mean = self.summary_df.pivot(
            index='faultNumber', 
            columns='model', 
            values='conditional_ARL1'
        )
        arl1_std = self.summary_df.pivot(
            index='faultNumber', 
            columns='model', 
            values='SDRL1'
        )
        
        # Combine mean ± std
        combined = pd.DataFrame(index=arl1_mean.index)
        for model in arl1_mean.columns:
            combined[model] = arl1_mean[model].apply(lambda x: f'{x:.1f}' if pd.notna(x) else 'N/A')
            combined[f'{model}_std'] = '±' + arl1_std[model].apply(lambda x: f'{x:.1f}' if pd.notna(x) else 'N/A')
        
        return combined
    
    def _create_arl0_table(self) -> pd.DataFrame:
        """Create ARL0 comparison table with mean and std."""
        arl0_mean = self.summary_df.pivot(
            index='faultNumber', 
            columns='model', 
            values='conditional_ARL0'
        )
        arl0_std = self.summary_df.pivot(
            index='faultNumber', 
            columns='model', 
            values='SDRL0'
        )
        
        # Combine mean ± std
        combined = pd.DataFrame(index=arl0_mean.index)
        for model in arl0_mean.columns:
            combined[model] = arl0_mean[model].apply(lambda x: f'{x:.1f}' if pd.notna(x) else 'N/A')
            combined[f'{model}_std'] = '±' + arl0_std[model].apply(lambda x: f'{x:.1f}' if pd.notna(x) else 'N/A')
        
        return combined
    
    def _create_detection_table(self) -> pd.DataFrame:
        """Create detection performance comparison table."""
        det_frac = self.summary_df.pivot(
            index='faultNumber', 
            columns='model', 
            values='avg_detection_fraction'
        )
        non_det = self.summary_df.pivot(
            index='faultNumber', 
            columns='model', 
            values='non_detection_fraction'
        )
        
        # Combine detection metrics
        combined = pd.DataFrame(index=det_frac.index)
        for model in det_frac.columns:
            combined[f'{model}_DetRate'] = (det_frac[model] * 100).apply(lambda x: f'{x:.1f}%')
            combined[f'{model}_MissRate'] = (non_det[model] * 100).apply(lambda x: f'{x:.1f}%')
        
        return combined
    
    def create_summary_statistics(self) -> pd.DataFrame:
        """
        Create overall summary statistics for each model.
        
        Returns:
            DataFrame with overall performance metrics
        """
        stats = []
        
        for model in self.models:
            model_data = self.df_results[self.df_results['model'] == model]
            model_summary = self.summary_df[self.summary_df['model'] == model]
            
            stats.append({
                'Model': model,
                'Mean ARL0': model_data['ARL0'].dropna().mean(),
                'Std ARL0': model_data['ARL0'].dropna().std(),
                'Mean ARL1': model_data['ARL1'].dropna().mean(),
                'Std ARL1': model_data['ARL1'].dropna().std(),
                'Overall Detection Rate': model_data['detection_fraction'].mean() * 100,
                'False Alarm Rate': (model_data['ARL0'].notna().sum() / len(model_data)) * 100,
                'Miss Rate': (model_data['ARL1'].isna().sum() / len(model_data)) * 100,
                #'Best Fault Detection': model_summary['conditional_ARL1'].idxmin()[1] if not model_summary['conditional_ARL1'].isna().all() else 'N/A',
                #'Worst Fault Detection': model_summary['conditional_ARL1'].idxmax()[1] if not model_summary['conditional_ARL1'].isna().all() else 'N/A'
            })
        
        summary_stats_df = pd.DataFrame(stats)
        
        # Round numerical columns
        numeric_cols = ['Mean ARL0', 'Std ARL0', 'Mean ARL1', 'Std ARL1', 
                       'Overall Detection Rate', 'False Alarm Rate', 'Miss Rate']
        summary_stats_df[numeric_cols] = summary_stats_df[numeric_cols].round(2)
        
        return summary_stats_df
    
    def plot_arl_comparison(self, figsize: Tuple[int, int] = (15, 6)) -> plt.Figure:
        """
        Create bar plots comparing ARL0 and ARL1 across models and faults.
        
        Returns:
            Matplotlib figure object
        """
        fig, axes = plt.subplots(1, 2, figsize=figsize)
        
        # ARL1 comparison
        arl1_pivot = self.summary_df.pivot(
            index='faultNumber', 
            columns='model', 
            values='conditional_ARL1'
        )
        arl1_pivot.plot(kind='bar', ax=axes[0], width=0.8)
        axes[0].set_title('ARL1 (Detection Delay) Comparison', fontsize=14, fontweight='bold')
        axes[0].set_xlabel('Fault Number', fontsize=12)
        axes[0].set_ylabel('ARL1 (samples)', fontsize=12)
        axes[0].legend(title='Model', bbox_to_anchor=(1.05, 1), loc='upper left')
        axes[0].grid(axis='y', alpha=0.3)
        axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45)
        
        # ARL0 comparison
        arl0_pivot = self.summary_df.pivot(
            index='faultNumber', 
            columns='model', 
            values='conditional_ARL0'
        )
        arl0_pivot.plot(kind='bar', ax=axes[1], width=0.8)
        axes[1].set_title('ARL0 (False Alarm) Comparison', fontsize=14, fontweight='bold')
        axes[1].set_xlabel('Fault Number', fontsize=12)
        axes[1].set_ylabel('ARL0 (samples)', fontsize=12)
        axes[1].legend(title='Model', bbox_to_anchor=(1.05, 1), loc='upper left')
        axes[1].grid(axis='y', alpha=0.3)
        axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45)
        
        plt.tight_layout()
        return fig
    
    def plot_detection_heatmap(self, figsize: Tuple[int, int] = (12, 8)) -> plt.Figure:
        """
        Create heatmap of detection rates across models and faults.
        
        Returns:
            Matplotlib figure object
        """
        fig, axes = plt.subplots(2, 1, figsize=figsize)
        
        # Detection fraction heatmap
        det_pivot = self.summary_df.pivot(
            index='faultNumber', 
            columns='model', 
            values='avg_detection_fraction'
        ) * 100  # Convert to percentage
        
        sns.heatmap(det_pivot.T, annot=True, fmt='.1f', cmap='RdYlGn', 
                   vmin=0, vmax=100, cbar_kws={'label': 'Detection Rate (%)'},
                   ax=axes[0])
        axes[0].set_title('Detection Rate Heatmap (Higher is Better)', fontsize=14, fontweight='bold')
        axes[0].set_xlabel('Fault Number', fontsize=12)
        axes[0].set_ylabel('Model', fontsize=12)
        
        # ARL1 heatmap (lower is better)
        arl1_pivot = self.summary_df.pivot(
            index='faultNumber', 
            columns='model', 
            values='conditional_ARL1'
        )
        
        sns.heatmap(arl1_pivot.T, annot=True, fmt='.0f', cmap='RdYlGn_r',
                   cbar_kws={'label': 'ARL1 (samples)'}, ax=axes[1])
        axes[1].set_title('ARL1 Heatmap (Lower is Better)', fontsize=14, fontweight='bold')
        axes[1].set_xlabel('Fault Number', fontsize=12)
        axes[1].set_ylabel('Model', fontsize=12)
        
        plt.tight_layout()
        return fig
    
    def plot_performance_boxplots(self, figsize: Tuple[int, int] = (15, 6)) -> plt.Figure:
        """
        Create boxplots showing distribution of ARL metrics across simulations.
        
        Returns:
            Matplotlib figure object
        """
        fig, axes = plt.subplots(1, 3, figsize=figsize)
        
        # ARL1 boxplot
        arl1_data = [self.df_results[self.df_results['model'] == model]['ARL1'].dropna() 
                     for model in self.models]
        bp1 = axes[0].boxplot(arl1_data, labels=self.models, patch_artist=True)
        axes[0].set_title('ARL1 Distribution', fontsize=14, fontweight='bold')
        axes[0].set_ylabel('ARL1 (samples)', fontsize=12)
        axes[0].set_xlabel('Model', fontsize=12)
        axes[0].grid(axis='y', alpha=0.3)
        
        # ARL0 boxplot
        arl0_data = [self.df_results[self.df_results['model'] == model]['ARL0'].dropna() 
                     for model in self.models]
        bp2 = axes[1].boxplot(arl0_data, labels=self.models, patch_artist=True)
        axes[1].set_title('ARL0 Distribution', fontsize=14, fontweight='bold')
        axes[1].set_ylabel('ARL0 (samples)', fontsize=12)
        axes[1].set_xlabel('Model', fontsize=12)
        axes[1].grid(axis='y', alpha=0.3)
        
        # Detection fraction boxplot
        det_data = [self.df_results[self.df_results['model'] == model]['detection_fraction'] * 100 
                   for model in self.models]
        bp3 = axes[2].boxplot(det_data, labels=self.models, patch_artist=True)
        axes[2].set_title('Detection Rate Distribution', fontsize=14, fontweight='bold')
        axes[2].set_ylabel('Detection Rate (%)', fontsize=12)
        axes[2].set_xlabel('Model', fontsize=12)
        axes[2].grid(axis='y', alpha=0.3)
        
        # Color the boxplots
        colors = plt.cm.Set3(np.linspace(0, 1, len(self.models)))
        for bp, color in zip([bp1, bp2, bp3], [colors, colors, colors]):
            for patch, c in zip(bp['boxes'], color):
                patch.set_facecolor(c)
                patch.set_alpha(0.7)
        
        plt.tight_layout()
        return fig
    
    def plot_fault_specific_comparison(self, fault_number: int, figsize: Tuple[int, int] = (12, 8)) -> plt.Figure:
        """
        Create detailed comparison for a specific fault.
        
        Args:
            fault_number: The fault number to analyze
            
        Returns:
            Matplotlib figure object
        """
        fault_data = self.df_results[self.df_results['faultNumber'] == fault_number]
        
        fig, axes = plt.subplots(2, 2, figsize=figsize)
        fig.suptitle(f'Fault {fault_number} - Detailed Comparison', fontsize=16, fontweight='bold')
        
        # ARL1 by simulation run
        for model in self.models:
            model_data = fault_data[fault_data['model'] == model]
            axes[0, 0].plot(model_data['simulationRun'], model_data['ARL1'], 
                          marker='o', label=model, alpha=0.7)
        axes[0, 0].set_title('ARL1 Across Simulation Runs')
        axes[0, 0].set_xlabel('Simulation Run')
        axes[0, 0].set_ylabel('ARL1')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # ARL0 by simulation run
        for model in self.models:
            model_data = fault_data[fault_data['model'] == model]
            axes[0, 1].plot(model_data['simulationRun'], model_data['ARL0'], 
                          marker='s', label=model, alpha=0.7)
        axes[0, 1].set_title('ARL0 Across Simulation Runs')
        axes[0, 1].set_xlabel('Simulation Run')
        axes[0, 1].set_ylabel('ARL0')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # Detection fraction comparison
        det_by_model = fault_data.groupby('model')['detection_fraction'].mean() * 100
        bars = axes[1, 0].bar(range(len(self.models)), det_by_model.values)
        axes[1, 0].set_xticks(range(len(self.models)))
        axes[1, 0].set_xticklabels(self.models)
        axes[1, 0].set_title('Average Detection Rate')
        axes[1, 0].set_ylabel('Detection Rate (%)')
        axes[1, 0].grid(axis='y', alpha=0.3)
        
        # Add value labels on bars
        for bar, value in zip(bars, det_by_model.values):
            axes[1, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                          f'{value:.1f}%', ha='center', va='bottom')
        
        # Summary statistics table
        axes[1, 1].axis('tight')
        axes[1, 1].axis('off')
        
        summary_data = []
        for model in self.models:
            model_data = fault_data[fault_data['model'] == model]
            summary_data.append([
                model,
                f"{model_data['ARL1'].dropna().mean():.1f}",
                f"{model_data['ARL0'].dropna().mean():.1f}",
                f"{model_data['detection_fraction'].mean()*100:.1f}%"
            ])
        
        table = axes[1, 1].table(cellText=summary_data,
                                colLabels=['Model', 'Mean ARL1', 'Mean ARL0', 'Det. Rate'],
                                cellLoc='center',
                                loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 1.5)
        
        plt.tight_layout()
        return fig
    
    def generate_full_report(self, save_path: str = None):
        """
        Generate a complete comparison report with all tables and plots.
        
        Args:
            save_path: Optional path to save the report figures
        """
        print("=" * 80)
        print("MODEL PERFORMANCE COMPARISON REPORT")
        print("=" * 80)
        print()
        
        # 1. Overall Summary Statistics
        print("1. OVERALL SUMMARY STATISTICS")
        print("-" * 40)
        summary_stats = self.create_summary_statistics()
        print(summary_stats.to_string(index=False))
        print()
        
        # 2. Comprehensive Comparison Table
        print("2. COMPREHENSIVE COMPARISON TABLE")
        print("-" * 40)
        comparison_table = self.create_comparison_table('all')
        print(comparison_table.to_string())
        print()
        
        # 3. ARL1 Detailed Table
        print("3. ARL1 COMPARISON (Mean ± Std)")
        print("-" * 40)
        arl1_table = self._create_arl1_table()
        print(arl1_table.to_string())
        print()
        
        # 4. Detection Performance Table
        print("4. DETECTION PERFORMANCE")
        print("-" * 40)
        det_table = self._create_detection_table()
        print(det_table.to_string())
        print()
        
        # Generate all plots
        fig1 = self.plot_arl_comparison()
        fig2 = self.plot_detection_heatmap()
        fig3 = self.plot_performance_boxplots()
        
        if save_path:
            fig1.savefig(f"{save_path}_arl_comparison.png", dpi=300, bbox_inches='tight')
            fig2.savefig(f"{save_path}_detection_heatmap.png", dpi=300, bbox_inches='tight')
            fig3.savefig(f"{save_path}_performance_boxplots.png", dpi=300, bbox_inches='tight')
            print(f"Figures saved to {save_path}_*.png")
        
        plt.show()
        
        return summary_stats, comparison_table


# Usage example:
def analyze_results(df_results: pd.DataFrame, summary_df: pd.DataFrame):
    """
    Main function to run the complete analysis.
    
    Args:
        df_results: Your results DataFrame
        summary_df: Your summary DataFrame
    """
    # Initialize analyzer
    analyzer = ModelComparisonAnalyzer(df_results, summary_df)
    
    # Generate full report
    summary_stats, comparison_table = analyzer.generate_full_report()
    
    # Generate fault-specific analysis for selected faults
    for fault in [1, 5, 10, 15, 20]:  # Analyze specific faults
        if fault in analyzer.fault_numbers:
            fig = analyzer.plot_fault_specific_comparison(fault)
            plt.show()
    
    # Return analyzer for further custom analysis if needed
    return analyzer

# After your main loop, use it like this:
analyzer = analyze_results(df_results, summary_df)